# 参数与超参数

**超参数**是指模型中无法通过学习得到，需要人工事先指定的模型参数。

在机器学习中，**调参**也即超参数调整是提升模型效果的关键步骤。

机器学习中的调参有
  - **手动调参**
  - **网格搜索（GridSearchCV）**
  - **随机搜索（RandomizedSearchCV）**
  - **贝叶斯搜索（BayesSearchCV）**

# 手动调参

**手动调参**是指通过人工调整模型的超参数，观察模型性能的变化，逐步找到最优参数组合的过程。

相比于自动调参（网格搜索GridSearchCV、随机搜索RandomizedSearchCV、贝叶斯搜索BayesSearchCV），手动调参耗费资源少、更加直观便捷。

## 手动调参步骤

1. 确定关键参数: 根据超参数作用确定要调整的超参数对象
2. 确定评估指标
    - 分类任务：准确率（accuracy）、F1分数（f1_score）、AUC-ROC（roc_auc_score）。
    - 回归任务：均方误差（MSE）、R²分数（r2_score）。
3. 训练模型并观察性能
    - 每次调整1~2个参数，记录训练集和验证集的得分。
    - 使用交叉验证（如cross_val_score）避免过拟合。
4. 手动调整参数
    - 如果模型欠拟合（训练集和验证集得分都低）：
        - 增加模型复杂度（如增大 max_depth、增加 n_estimators）。
        - 减少正则化（如减小 min_samples_split）。
    - 如果模型过拟合（训练集得分高，验证集得分低）：
        - 降低模型复杂度（如减小 max_depth）。
        - 增加正则化（如增大 min_samples_split）
5. 重复以上步骤直至效果满意

## 手动调参技巧

- 先调重要参数（如随机森林的 max_depth、n_estimators）。
- 每次只调1个参数，避免混淆影响。
- 记录每次实验的结果，便于回溯分析。
- 结合学习曲线判断欠拟合/过拟合。
- 最终用测试集验证，避免数据泄露。

# 网格搜索GridSearchCV

**GridSearchCV**通过**网格搜索（Grid Search）结合交叉验证（Cross Validation）**的方式，系统地遍历多个参数组合，从中找到使模型性能最优的那一组超参数。

具体来说，网格搜索将每个超参数的可能取值组合成一个网格，然后穷举搜索所有可能的组合，对每个组合进行交叉验证，并评估模型在验证集上的性能。最终选择在验证集上性能最佳的超参数组合作为最终的模型参数。

## 网格搜索GridSearchCV优缺点

- 优点
    - 简单易用
    - 在提供的网格内保证能够找到“全局最优”的一组超参数
    - 结合交叉验证，鲁棒性强
- 缺点
    - 参数组合多或数据集大时耗费计算开销大
    - 只适用于离散搜索空间（无法自动缩放或自适应调整）

## GridSearchCV()

- `estimator`: 要被调参的模型对象
- `param_grid`: 一个字典或字典列表，定义参数搜索空间
- `scoring`: 模型评估标准
- `n_jobs`: 并行计算使用的 CPU 核心数
    - n_jobs=1: 单线程
    - n_jobs=-1: 使用全部 CPU 核心（推荐）
- `cv`: 设置交叉验证方式
    - None：默认 5 折交叉验证
    - 整数：表示 K 折。如 cv=10表示十折
    - 对象：KFold、StratifiedKFold（参考C04课程讲解）
- `verbose`: 控制日志输出信息量
    - 0: 不输出
    - 1: 输出基本信息
    - 2: 更详细（推荐用于调试）

## 返回结果
- `.best_params_`：最佳参数组合
- `.best_score_`：最佳得分（交叉验证）
- `.best_estimator_`：最优模型对象（如果 refit=True）
- `.cv_results_`：包含所有搜索结果的详细信息字典，可转换为 DataFrame

## Iris鸢尾花数据集

Iris鸢尾花数据集是一个经典的机器学习数据集，包含了三种鸢尾花（Setosa, Versicolor, Virginica）的不同特征，每个花种有 50 个样本，总共有 150 个样本
- sepal_length：花萼长度
- sepal_width：花萼宽度
- petal_length：花瓣长度
- petal_width：花瓣宽度
- species：花种

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

iris = load_iris() # 加载鸢尾花数据集
X = iris.data # 自变量
y = iris.target  # 因变量
feature_names = iris.feature_names # 特征名称
iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

## 代码实现

In [8]:
# 导入必要的库
from sklearn.ensemble import RandomForestClassifier  # 随机森林分类器
from sklearn.datasets import load_iris  # 鸢尾花数据集
from sklearn.model_selection import train_test_split, GridSearchCV  # 数据分割和网格搜索
from sklearn.metrics import classification_report  # 分类评估报告

# 1. 加载数据
# ------------------------------------------------
# 使用sklearn自带的鸢尾花数据集（150个样本，4个特征，3个类别）
data = load_iris()
X, y = data.data, data.target  # X是特征矩阵，y是目标标签

# 2. 划分训练集和验证集
# ------------------------------------------------
# 随机拆分数据，验证集占20%，random_state保证每次运行结果一致
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42
)

# 3. 定义参数网格
# ------------------------------------------------
# 指定需要搜索的超参数组合：
# - n_estimators: 树的数量 [50, 100, 200]
# - max_depth: 树的最大深度 [3, 5, 7]
# - min_samples_split: 节点分裂所需最小样本数 [2, 4]
# - 'max_features': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
# 共 3 * 3 * 2 * 9 = 162 种组合
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 4],
    'max_features': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
}

# 4. 初始化GridSearchCV
# ------------------------------------------------
grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),  # 基础模型
    param_grid=param_grid,  # 参数网格
    cv=5,                  # 5折交叉验证
    scoring='accuracy',    # 评估指标（准确率）
    n_jobs=-1              # 使用所有CPU核心并行计算
)

# 5. 执行网格搜索
# ------------------------------------------------
# 对18种参数组合分别进行5折交叉验证，共162 * 5=810次训练
grid_search.fit(X_train, y_train)

# 6. 输出最优结果
# ------------------------------------------------
print("Best parameters:", grid_search.best_params_)  # 最佳参数组合
print("Best cross-val score: {:.4f}".format(grid_search.best_score_))  # 最佳交叉验证准确率

# 7. 在测试集上评估
# ------------------------------------------------
# 用最优参数的模型预测验证集
y_pred = grid_search.predict(X_test)

# 打印分类报告（包含精确率、召回率、F1值等）
# - precision: 精确率（预测为正例中实际为正的比例）
# - recall: 召回率（实际为正例中被正确预测的比例）
# - f1-score: 精确率和召回率的调和平均
# - support: 各类别样本数
print(classification_report(y_test, y_pred))

Best parameters: {'max_depth': 3, 'max_features': 0.5, 'min_samples_split': 2, 'n_estimators': 50}
Best cross-val score: 0.9500
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



# 随机搜索RandomizedSearchCV

**RandomizedSearchCV** 是 scikit-learn 提供的一种超参数优化方法，用于在给定的参数空间中随机采样并评估模型性能

- **核心原理**
    - **随机采样**：从参数分布（如均匀分布、离散列表）中抽取固定次数的参数组合，而非穷举所有可能（网格搜索）。
    - **交叉验证**：对每组参数使用交叉验证（如 5 折）评估模型性能。
    - **返回最佳参数**：选择交叉验证得分最高的参数组合。

## 随机搜索RandomizedSearchCV优缺点

- 优点
    - 计算效率高：相比网格搜索（GridSearchCV），随机搜索不穷举所有参数组合，而是通过随机采样（n_iter 次）减少计算量，特别适合高维参数空间或训练耗时的模型（如深度学习、GBDT）。
    - 避免局部最优陷阱：尤其在参数间存在非线性交互时（如神经网络的层数与学习率）， 随机搜索RandomizedSearchCV的随机性可帮助跳出局部最优解。
- 缺点：
    - 可能错过全局最优：依赖随机采样，若 n_iter 过小或参数分布设计不合理，可能漏掉关键参数组合（例如网格搜索能找到但随机搜索未抽样的点）。

## RandomizedSearchCV()

- `estimator`: 要被调参的模型对象
- `param_distributions`: 参数空间字典，定义参数搜索空间
- `n_iter`: 随机采样的参数组合数量（越大越可能找到最优解，但计算成本越高）
- `scoring`: 模型评估标准
- `n_jobs`: 并行计算使用的 CPU 核心数
    - n_jobs=1: 单线程
    - n_jobs=-1: 使用全部 CPU 核心（推荐）
- `cv`: 设置交叉验证方式
    - None：默认 5 折交叉验证
    - 整数：表示 K 折。如 cv=10表示十折
    - 对象：KFold、StratifiedKFold（参考C04课程讲解）
- `verbose`: 控制日志输出信息量
    - 0: 不输出
    - 1: 输出基本信息
    - 2: 更详细（推荐用于调试）
- `random_state`: 随机数种子（保证结果可复现）

## 返回结果
- `.best_params_`：最佳参数组合
- `.best_score_`：最佳得分（交叉验证）
- `.best_estimator_`：最优模型对象（如果 refit=True）
- `.cv_results_`：包含所有搜索结果的详细信息字典，可转换为 DataFrame

## 代码实现

In [9]:
# 导入必要的库
from sklearn.ensemble import RandomForestClassifier  # 随机森林分类器
from sklearn.datasets import load_iris  # 鸢尾花数据集
from sklearn.model_selection import train_test_split, RandomizedSearchCV  # 数据分割和随机搜索
from sklearn.metrics import classification_report  # 分类评估报告
from scipy.stats import randint, uniform  # 用于定义参数的概率分布
import warnings
warnings.filterwarnings('ignore')  # 忽略警告信息

# 1. 加载数据
# ------------------------------------------------
# 使用sklearn自带的鸢尾花数据集（150个样本，4个特征，3个类别）
data = load_iris()
X, y = data.data, data.target  # X是特征矩阵，y是目标标签

# 2. 划分训练集和验证集
# ------------------------------------------------
# 随机拆分数据，验证集占20%，random_state保证每次运行结果一致
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42
)

# 3. 定义参数分布
# ------------------------------------------------
param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 4],
    'max_features': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
}

# 4. 初始化RandomizedSearchCV
# ------------------------------------------------
random_search = RandomizedSearchCV(
    estimator=RandomForestClassifier(random_state=42),  # 基础模型
    param_distributions=param_dist,  # 参数分布
    n_iter=20,                       # 随机采样20组参数组合
    cv=5,                            # 5折交叉验证
    scoring='accuracy',              # 评估指标（准确率）
    random_state=42,                 # 保证结果可复现
    n_jobs=-1                        # 使用所有CPU核心并行计算
)

# 5. 执行随机搜索
# ------------------------------------------------
# 对随机采样的20种参数组合分别进行5折交叉验证，共20 * 5=100次训练
random_search.fit(X_train, y_train)

# 6. 输出最优结果
# ------------------------------------------------
print("Best parameters:", random_search.best_params_)  # 最佳参数组合
print("Best cross-val score: {:.4f}".format(random_search.best_score_))  # 最佳交叉验证准确率

# 7. 在验证集上评估
# ------------------------------------------------
# 用最优参数的模型预测验证集
y_pred = random_search.predict(X_test)

# 打印分类报告（包含精确率、召回率、F1值等）
print(classification_report(y_test, y_pred))

Best parameters: {'n_estimators': 200, 'min_samples_split': 2, 'max_features': 0.9, 'max_depth': 7}
Best cross-val score: 0.9500
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



# 贝叶斯搜索BayesSearchCV

**贝叶斯搜索BayesSearchCV** 是 scikit-optimize（skopt）库提供的基于贝叶斯优化的超参数搜索方法，相比 GridSearchCV 和 RandomizedSearchCV，它通过​​自适应参数采样​​更高效地找到最优解，特别适合​​计算成本高​​的模型调优。

贝叶斯搜索BayesSearchCV原理：
1. **建立概率模型**：用高斯过程（Gaussian Process）或树形Parzen估计器（TPE）建模目标函数（如交叉验证得分）与超参数的关系。
2. **迭代优化**：
    - 根据已有参数组合的表现，预测哪些区域更可能包含最优解。
    - 优先探索表现好或不确定性高的参数区域。
3. **收敛**：经过多次迭代后，参数采样集中在最优解附近。

## 贝叶斯搜索BayesSearchCV优缺点

- 优点：
    - 高效调参：通过自适应采样（如高斯过程或TPE）优先探索表现好的参数区域，比随机搜索/网格搜索更快收敛到最优解。
    - 减少计算浪费：避免网格搜索的“盲目穷举”和随机搜索的“低效随机”，尤其适合训练成本高的模型（如深度学习、XGBoost）。
- 缺点：
    - 单次迭代成本高：每轮迭代需更新概率模型，计算开销比随机搜索大（适合“训练耗时 >> 搜索耗时”的场景）
    - 对初始点敏感：若初始随机采样点质量差，可能陷入局部最优（可通过增加 n_iter 缓解）

## BayesSearchCV()

- `estimator`: 要被调参的模型对象
- `search_spaces`: 参数空间字典，定义参数搜索空间
- `n_iter`: 参数组合数量（越大越可能找到最优解，但计算成本越高）
- `scoring`: 模型评估标准
- `n_jobs`: 并行计算使用的 CPU 核心数
    - n_jobs=1: 单线程
    - n_jobs=-1: 使用全部 CPU 核心（推荐）
- `cv`: 设置交叉验证方式
    - None：默认 5 折交叉验证
    - 整数：表示 K 折。如 cv=10表示十折
    - 对象：KFold、StratifiedKFold（参考C04课程讲解）
- `verbose`: 控制日志输出信息量
    - 0: 不输出
    - 1: 输出基本信息
    - 2: 更详细（推荐用于调试）
- `random_state`: 随机数种子（保证结果可复现）

## 返回结果
- `.best_params_`：最佳参数组合
- `.best_score_`：最佳得分（交叉验证）
- `.best_estimator_`：最优模型对象（如果 refit=True）
- `.cv_results_`：包含所有搜索结果的详细信息字典，可转换为 DataFrame

## 代码实现

In [10]:
# 导入必要的库
from sklearn.ensemble import RandomForestClassifier  # 随机森林分类器
from sklearn.datasets import load_iris  # 鸢尾花数据集
from sklearn.model_selection import train_test_split  # 数据分割
from sklearn.metrics import classification_report  # 分类评估报告
from skopt import BayesSearchCV  # 贝叶斯优化搜索（需安装scikit-optimize）
from skopt.space import Real, Integer, Categorical  # 参数空间定义
import warnings
warnings.filterwarnings('ignore')  # 忽略警告信息

# 1. 加载数据
# ------------------------------------------------
# 使用sklearn自带的鸢尾花数据集（150个样本，4个特征，3个类别）
data = load_iris()
X, y = data.data, data.target  # X是特征矩阵，y是目标标签

# 2. 划分训练集和验证集
# ------------------------------------------------
# 随机拆分数据，验证集占20%，random_state保证每次运行结果一致
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42
)

# 3. 定义贝叶斯搜索的参数空间
param_space = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 4],
    'max_features': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
}

# 4. 初始化BayesSearchCV
# ------------------------------------------------
bayes_search = BayesSearchCV(
    estimator=RandomForestClassifier(random_state=42),  # 基础模型
    search_spaces=param_space,  # 参数空间
    n_iter=20,                  # 迭代次数（即参数评估次数）
    cv=5,                       # 5折交叉验证
    scoring='accuracy',         # 评估指标（准确率）
    random_state=42,            # 保证结果可复现
    n_jobs=-1,                  # 使用所有CPU核心
    optimizer_kwargs={'base_estimator': 'GP'}  # 使用高斯过程建模目标函数
)

# 5. 执行贝叶斯优化
# ------------------------------------------------
print("开始贝叶斯优化...")
# 过程包含：
# 1) 初始随机采样若干点
# 2) 用高斯过程建模参数->得分关系
# 3) 根据模型预测，选择最有潜力的新参数评估
bayes_search.fit(X_train, y_train)
print("优化完成！")

# 6. 输出最优结果
# ------------------------------------------------
print("\nBest parameters:", bayes_search.best_params_)  # 最佳参数组合
print("Best cross-val score: {:.4f}".format(bayes_search.best_score_))  # 最佳交叉验证得分

# 7. 在测试集上评估
# ------------------------------------------------
y_pred = bayes_search.predict(X_test)
print("\n测试集性能报告:")
# 输出分类报告（精确率/召回率/F1值）
print(classification_report(y_test, y_pred))

# 8. 查看所有尝试的参数组合（可选）
# ------------------------------------------------
import pandas as pd
results = pd.DataFrame(bayes_search.cv_results_)  # 转为DataFrame
print("\n参数组合:")
# 按得分排序
results[['params', 'mean_test_score']].sort_values('mean_test_score', ascending=False)

开始贝叶斯优化...
优化完成！

Best parameters: OrderedDict({'max_depth': 5, 'max_features': 0.7, 'min_samples_split': 4, 'n_estimators': 50})
Best cross-val score: 0.9500

测试集性能报告:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30


参数组合:


,params,mean_test_score
0,"{'max_depth': 5, 'max_features': 0.7, 'min_sam...",0.950000
2,"{'max_depth': 5, 'max_features': 0.9, 'min_sam...",0.950000
17,"{'max_depth': 5, 'max_features': 0.6, 'min_sam...",0.950000
5,"{'max_depth': 7, 'max_features': 0.9, 'min_sam...",0.950000
6,"{'max_depth': 5, 'max_features': 0.7, 'min_sam...",0.950000
7,"{'max_depth': 5, 'max_features': 0.9, 'min_sam...",0.950000
8,"{'max_depth': 7, 'max_features': 0.7, 'min_sam...",0.950000
9,"{'max_depth': 3, 'max_features': 0.8, 'min_sam...",0.950000
1,"{'max_depth': 7, 'max_features': 0.8, 'min_sam...",0.950000
11,"{'max_depth': 7, 'max_features': 0.6, 'min_sam...",0.950000


# 总结

| 特性               | GridSearchCV                          | RandomizedSearchCV                     | BayesSearchCV                          |
|--------------------|---------------------------------------|----------------------------------------|----------------------------------------|
| 搜索策略           | 穷举所有参数组合                      | 随机采样参数组合                       | 基于概率模型自适应采样（如高斯过程）   |
| 参数空间类型       | 仅离散（需显式列举）                  | 离散 + 连续（支持分布抽样）            | 离散 + 连续 + 混合（支持概率分布）      |
| 计算效率           | ❌ 低效（参数组合数爆炸）             | ✅ 中等（通过 n_iter 控制）            | ✅✅ 高效（智能收敛，减少无效尝试）     |
| 是否找到全局最优   | ✅ 在给定网格内保证最优               | ❌ 可能错过最优（依赖随机抽样）        | ⚠️ 接近最优（依赖初始点和迭代次数）    |
| 并行化支持         | ✅ 支持（n_jobs）                     | ✅ 支持（n_jobs）                      | ✅ 支持（n_jobs）                       |
| 实现库             | scikit-learn 内置                     | scikit-learn 内置                     | 需 scikit-optimize（非 sklearn 官方）  |
| 最佳参数返回值     | best_params_                          | best_params_                          | best_params_                           |
| 主要优点           | 结果确定性强                          | 适合高维参数空间                      | 计算资源利用率高，适合复杂参数交互     |
| 主要缺点           | 计算成本高，不适合连续参数            | 结果可能不稳定                        | 单次迭代成本高，需额外依赖库           |
| 典型应用场景       | 参数少（<5）且可穷举                  | 快速原型开发/中等规模调参             | 高计算成本模型（如深度学习、XGBoost）  |